# Storing Conversation History

Storing conversation history in a Neo4j database allows you to analyze the conversation history to understand trends and improve outcomes.


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from uuid import uuid4
import os

You must create and assign a session ID to each conversation to identify them.

The session ID can be any unique value, such as a `Universally` Unique Identifier (UUID)`.

You can generate a random UUID using the Python uuid.uuid4 function.

Create a new `SESSION_ID` constant in your chat model program.

In [ ]:
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")


#### Create a OpenAI chat model instance

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
chat_llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

#### Connecting to a Neo4j instance

The following code will connect to a Neo4j database and run a simple query.

In [ ]:
graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="12345678"
    )

Session ID: 584de8a2-8099-4967-92f3-e24f8204376a


This session ID will be used to identify the conversation in Neo4j.

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

Previously, the get_memory function returned an instance of ChatMessageHistory.

The get_memory function should now return an instance of Neo4jChatMessageHistory. You should pass the session_id and the graph connection you created as parameters.

In [6]:
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

In [7]:
chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

Finally, you must add the SESSION_ID to the config when you invoke the chat model.

In [8]:
current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while True:
    question = input("> ")

    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": SESSION_ID}
        }
    )
    
    print(response)

Dude, the surf at Watergate Bay is 3ft with onshore winds. It's not epic, but you can still catch some fun waves!
Bro, the surf at Watergate Bay is 3ft with onshore winds. It's not firing, but you can still shred some gnarly waves!
Sorry, my bad, bro! You're at Watergate Bay with 3ft waves and onshore winds. It's not the best conditions, but you can still hang ten and catch some waves, dude!
Cowabunga, dude! If you're still out there, catch some sick waves and have a rad time on the water!
Catch you later, dude! Keep stoked and ride those waves like a boss!
Catch you on the flip side, dude! Stay stoked and keep on shredding those waves!


KeyboardInterrupt: 